In [17]:
import sys
sys.path.append('../')

!pwd

/c/Users/Vladimir/Desktop/Everything/Work/ARLIS/development/incubator-flagon-distill/Data Generation - Vlad


In [18]:
!ls

Log Data Extraction Example.ipynb
logs_1682015023633.json


In [19]:
import datetime
import distill
import json
import networkx as nx
import os
import pandas as pd
import plotly.express as px
import re

df = pd.read_json("logs_1682015023633.json")
df.head()

,target,path,pageUrl,pageTitle,pageReferrer,browser,clientTime,microTime,location,scrnRes,...,logType,userAction,details,userId,toolVersion,toolName,useraleVersion,sessionID,attributes,style
0,Window,[Window],http://localhost:8088/superset/dashboard/world...,World Bank's Data,http://localhost:8088/login/,"{'browser': 'chrome', 'version': '112.0.0'}",1682013566400,0.2,"{'x': None, 'y': None}","{'width': 1397, 'height': 865}",...,custom,True,"{'width': 1553, 'height': 1040}",nobody,NaN,test_app,2.3.0,session_1682012031278,{},{}
1,Window,[Window],http://localhost:8088/superset/dashboard/world...,World Bank's Data,http://localhost:8088/login/,"{'browser': 'chrome', 'version': '112.0.0'}",1682013567125,0.6,"{'x': None, 'y': None}","{'width': 1397, 'height': 872}",...,custom,True,"{'width': 1553, 'height': 1047}",nobody,NaN,test_app,2.3.0,session_1682012031278,{},{}
2,div.dashboard-content css-1xmvqbs,"[div.dashboard-content css-1xmvqbs, div.dashbo...",http://localhost:8088/superset/dashboard/world...,World Bank's Data,http://localhost:8088/login/,"{'browser': 'chrome', 'version': '112.0.0'}",1682014305418,0.8,"{'x': 543, 'y': 127}","{'width': 1397, 'height': 872}",...,custom,True,"{'clicks': 1, 'ctrl': False, 'alt': False, 'sh...",nobody,NaN,test_app,2.3.0,session_1682012031278,{'class': 'dashboard-content css-1xmvqbs'},{}
3,svg.[object SVGAnimatedString],"[svg.[object SVGAnimatedString], span.anticon ...",http://localhost:8088/superset/dashboard/world...,World Bank's Data,http://localhost:8088/login/,"{'browser': 'chrome', 'version': '112.0.0'}",1682014316357,0.4,"{'x': 224, 'y': 67}","{'width': 1397, 'height': 872}",...,custom,True,"{'clicks': 1, 'ctrl': False, 'alt': False, 'sh...",nobody,NaN,test_app,2.3.0,session_1682012031278,"{'width': '1em', 'height': '1em', 'viewBox': '...",{}
4,svg.[object SVGAnimatedString],"[svg.[object SVGAnimatedString], div.superset-...",http://localhost:8088/superset/dashboard/world...,World Bank's Data,http://localhost:8088/login/,"{'browser': 'chrome', 'version': '112.0.0'}",1682015026480,0.7,"{'x': 550, 'y': 538}","{'width': 1397, 'height': 872}",...,custom,True,"{'clicks': 1, 'ctrl': False, 'alt': False, 'sh...",nobody,NaN,test_app,2.3.0,session_1682012031278,"{'width': 730, 'data-width': 730, 'class': 'da...","{'overflow-x': 'hidden', 'overflow-y': 'hidden'}"


In [20]:
print(df.columns)
print( type(df['attributes']) )
print( type(df['attributes'].iloc[0]) )
print( (df['attributes'].iloc[-1]['data-info']['name']) )

Index(['target', 'path', 'pageUrl', 'pageTitle', 'pageReferrer', 'browser',
       'clientTime', 'microTime', 'location', 'scrnRes', 'type', 'logType',
       'userAction', 'details', 'userId', 'toolVersion', 'toolName',
       'useraleVersion', 'sessionID', 'attributes', 'style'],
      dtype='object')
<class 'pandas.core.series.Series'>
<class 'dict'>
South Africa


As we can see, the logs json produces a dataframe with a column that hold each web-interaction's (in this case, exclusively clicks') dictionary of attributes wherein the name of the clicked country is located

Now lets try extracting the relevant logs into list or Segment types. My silly list way:

In [21]:
sessions = []
curr_session = []
for index, row in df.iterrows(): # for each row
    attr = row['attributes']
    # if the attributes column element is a dictionary and contains the data-info dictionary, append the click log
    if isinstance(attr, dict) and ('data-info' in attr):
        curr_session.append( row )
    # Otherwise, if the session is over, append the last session and reset for the new one
    elif len(curr_session)!=0:
        sessions.append(curr_session)
        curr_session = []
print("Final # of sessions: ", len(sessions))

Final # of sessions:  24


The Distill-supported Segment creation way

In [29]:
import sys
sys.path.append('../')

import datetime
import distill
import json
import networkx as nx
import os
import pandas as pd
import plotly.express as px
import re

def setup(file, date_type):
    with open(file) as json_file:
        raw_data = json.load(json_file)

    data = {}
    for log in raw_data:
        data[distill.getUUID(log)] = log
        
    # Convert clientTime to specified type
    for uid in data:
        log = data[uid]
        client_time = log['clientTime']
        if date_type == "integer":
            log['clientTime'] = distill.epoch_to_datetime(client_time)
        elif date_type == "datetime":
            log['clientTime'] = pd.to_datetime(client_time, unit='ms', origin='unix')

    # Sort
    sorted_data = sorted(data.items(), key=lambda kv: kv[1]['clientTime'])
    sorted_dict = dict(sorted_data)

    return (sorted_data, sorted_dict)

data_many_session = setup("logs_1682015023633.json", "datetime")
sorted_dict = data_many_session[1]

# Create segments based on sessionID
segments = []
segments = distill.Segments()
session_ids = sorted(distill.find_meta_values('sessionID', sorted_dict), key=lambda sessionID: sessionID)
for session_id in session_ids:
    segments.append_segments(distill.generate_collapsing_window_segments(sorted_dict, 'sessionID', [session_id], session_id))
    
# Improve readability of Segment names
for index in range(len(segments)):
    segments[index].segment_name = "Session" + str(index)
    
for segment in segments:
    print(segment.get_segment_name() + "   Length: " + str(segment.get_num_logs()) + "   Start Time: " + str(segment.get_start_end_val()[0]))

Session0   Length: 605   Start Time: 2023-04-20 17:59:26.400000
Session1   Length: 605   Start Time: 2023-04-20 17:59:26.400000
Session2   Length: 605   Start Time: 2023-04-20 17:59:26.400000
Session3   Length: 605   Start Time: 2023-04-20 17:59:26.400000
Session4   Length: 605   Start Time: 2023-04-20 17:59:26.400000
Session5   Length: 605   Start Time: 2023-04-20 17:59:26.400000
Session6   Length: 605   Start Time: 2023-04-20 17:59:26.400000
Session7   Length: 605   Start Time: 2023-04-20 17:59:26.400000


Why are all logs part of a single Segment, despite periodic page refreshes?